In [10]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
import numpy as np
#from sklearn.preprocessing import LabelEncoder
from keras.optimizers import Adam ,SGD
from keras.layers import Input, Dense, Convolution2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
import glob,matplotlib as plt
from keras.applications.vgg16 import VGG16, preprocess_input,decode_predictions
from keras.applications.resnet50 import ResNet50,preprocess_input, decode_predictions



def get_compile_model():
    img_width, img_height = 224, 224
    # model = applications.VGG16(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
    model = applications.ResNet50(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

    # Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
#     for layer in model.layers[:16]:
#         layer.trainable = False
#     for layer in model.layers[16:]:
#         layer.trainable = True
        
#     for layer in model.layers:
#         layer.trainable = False
    # Adding custom Layers
    x = model.output
    #print(x.shape)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation="relu")(x)
    ## 4 as the number of categories
    predictions = Dense(4, activation="softmax")(x)

    # creating the final model
    model_final = Model(input=model.input, output=predictions)
    # model.load_weights('caavo-1.h5')
    # print(model.summary())
    adam = Adam(lr=1e-3)
    model_final.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model_final


In [12]:
import os
import cv2
import keras
import numpy as np
from keras.utils import Sequence
from keras.utils import np_utils


class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args:
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """

    def __init__(self, directory, batch_size=1, shuffle=True, data_augmentation=True):
        # Initialize the params
        self.batch_size = batch_size
        self.directory = directory
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data()
        # Print basic statistics information
        self.print_stats()


    def search_data(self):
        X_path = []
        Y_dict = {}
        # list all kinds of sub-folders
        self.dirs = sorted(os.listdir(self.directory))
        one_hots = np_utils.to_categorical(range(len(self.dirs)))
        for i, folder in enumerate(self.dirs):
            folder_path = os.path.join(self.directory, folder)
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                # append the each file path, and keep its label
                X_path.append(file_path)
                Y_dict[file_path] = one_hots[i]
        return X_path, Y_dict

    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = len(self.dirs)
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files, self.n_classes))
        for i, label in enumerate(self.dirs):
            print('%10s : ' % (label), i)
        return None

    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def read_image(self,img_path, H, W):
        #print(f"in read img\n{img_path}\n")
        # img_path=img_path+".jpg"
        #print(f"in read img--{img_path}")
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)

        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (W, H))  # you can resize to  (128,128) or (256,256)

        return img

    def data_generation(self, batch_path):
        #print(f"in data generation\n {batch_path}\n")
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y

    def load_data(self, path):
        # load the processed .npy files which have 5 channels (1-3 for RGB, 4-5 for optical flows)
        #data = np.load(path, mmap_mode='r')
        #print(f"[+][+] in load data:path-->{path}]n")
        data = self.read_image(path, 224, 224)
        data = np.float32(data)
        return data


#################################################################3
import keras.backend as K
from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    if epoch % 10 == 0 and epoch != 0:
    #if epoch  == 1 and epoch != 0:
    #print("---------------in scheduler-----------------------")
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.7)
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)    

model =get_compile_model()
#from keras.models import load_model

#path=r'/home/jupyter/training_data/weights/weights_at_epoch_1.h5'
#model = model.load_weights(r'/home/jupyter/training_data/weights/weights_at_epoch_1.h5')

from keras.callbacks import ModelCheckpoint, CSVLogger

class MyCbk(keras.callbacks.Callback):

    def __init__(self, model):
         self.model_to_save = model

    def on_epoch_end(self, epoch, logs=None):
        self.model_to_save.save(r'/home/jupyter/training_data/model/model_at_epoch_%d.h5' % (epoch+1))
        self.model_to_save.save_weights(r'/home/jupyter/training_data/weights/weights_at_epoch_%d.h5' % (epoch+1))

check_point = MyCbk(model)


filename = r'/home/jupyter/training_data/log/ours_log.csv'
csv_logger = CSVLogger(filename, separator=',', append=True)
callbacks_list = [check_point, csv_logger, reduce_lr]




path_train = r'/home/jupyter/Train'
path_val = r'/home/jupyter/Val'
num_epochs=5
batch_size=100
checkpoint_dir=""
num_workers=8

train_generator = DataGenerator(directory=path_train,batch_size=batch_size,data_augmentation=False)

val_generator = DataGenerator(directory=path_val, batch_size=batch_size,data_augmentation=False)




hist = model.fit_generator(
    generator=train_generator,
    validation_data=val_generator,
    callbacks=callbacks_list,
    verbose=1,
    epochs=num_epochs,
    workers=num_workers ,
    max_queue_size=4,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator))

/opt/conda/lib/python3.7/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Found 20934 files belonging to 4 classes.
Blouses_Shirts :  0
   Dresses :  1
    Shorts :  2
    Skirts :  3
Found 7661 files belonging to 4 classes.
Blouses_Shirts :  0
   Dresses :  1
    Shorts :  2
    Skirts :  3
Epoch 1/5
210/210 [==============================] - 480s 2s/step - loss: 0.6764 - accuracy: 0.7285 - val_loss: 1.8730 - val_accuracy: 0.5169
Epoch 2/5
210/210 [==============================] - 451s 2s/step - loss: 0.4718 - accuracy: 0.8123 - val_loss: 0.5569 - val_accuracy: 0.7148
Epoch 3/5
210/210 [==============================] - 451s 2s/step - loss: 0.4152 - accuracy: 0.8322 - val_loss: 0.3968 - val_accuracy: 0.7710
Epoch 4/5
210/210 [==============================] - 451s 2s/step - loss: 0.3697 - accuracy: 0.8501 - val_loss: 1.1818 - val_accuracy: 0.7259
Epoch 5/5
210/210 [==============================] - 451s 2s/step - loss: 0.3215 - accuracy: 0.8691 - val_loss: 0.4634 - val_accuracy: 0.7495
